# 1. Introdução

A Amazon Web Services (AWS) é a plataforma de nuvem mais adotada e mais abrangente do mundo, oferecendo mais de 200 serviços completos de datacenters em todo o mundo. Milhões de clientes, incluindo as startups de crescimento mais rápido, grandes empresas e os maiores órgãos governamentais, estão usando a AWS para reduzirem seus custos, ficarem mais ágeis e inovarem mais rapidamente.

O ambiente AWS permite o processamento e análise de dados direto na nuvem, que implica principalmente em uma redução significativa de tempo de processamento e reduação de espaço fisícos para armazenamento de dados.

Este script tem como objetivo uma análise preditiva relacionada a preços de casas nos E.U.A utilizada algoritmos de regressão.

# 2. Carga dos Dados

## 2.1 Instalação dos Pacotes

In [2]:
#%conda update -n base conda
#%conda install numpy
#%conda install pandas
#%conda install seaborn
#%conda install matplotlib
#%conda install scikit-learn
#%conda install sagemaker
#%conda install aws-c-io

### 2.2 Pacotes carregados no script

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')

%matplotlib inline

# 3. Tratamento de Bases

In [19]:
# Carga da base de dados
base_casas = pd.read_csv('house_prices.csv')
base_casas.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [20]:
# Remoção de colunas que não serão usadas
base_casas.drop(columns = ['id', 'date', 'sqft_living15', 'sqft_lot15'], axis = 1, inplace = True)
base_casas

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,360000.0,3,2.50,1530,1131,3.0,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346
21609,400000.0,4,2.50,2310,5813,2.0,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362
21610,402101.0,2,0.75,1020,1350,2.0,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299
21611,400000.0,3,2.50,1600,2388,2.0,0,0,3,8,1600,0,2004,0,98027,47.5345,-122.069


In [21]:
# Divisão de bases em treino e teste
base_treinamento = base_casas.iloc[0:15129, :]
base_treinamento.shape

(15129, 17)

In [22]:
# Divisão de bases em treino e teste
base_teste = base_casas.iloc[15129 :, :]
base_teste.shape

(6484, 17)

In [23]:
# Cria variáveis x e y para treino e teste
x_teste = base_teste.iloc[:,1:17].values
x_teste

array([[ 4.00000e+00,  2.75000e+00,  2.58000e+03, ...,  9.81150e+04,
         4.76753e+01, -1.22304e+02],
       [ 4.00000e+00,  2.50000e+00,  3.20000e+03, ...,  9.80590e+04,
         4.75273e+01, -1.22143e+02],
       [ 3.00000e+00,  1.00000e+00,  8.40000e+02, ...,  9.81780e+04,
         4.74940e+01, -1.22275e+02],
       ...,
       [ 2.00000e+00,  7.50000e-01,  1.02000e+03, ...,  9.81440e+04,
         4.75944e+01, -1.22299e+02],
       [ 3.00000e+00,  2.50000e+00,  1.60000e+03, ...,  9.80270e+04,
         4.75345e+01, -1.22069e+02],
       [ 2.00000e+00,  7.50000e-01,  1.02000e+03, ...,  9.81440e+04,
         4.75941e+01, -1.22299e+02]])

In [24]:
# Cria variáveis x e y para treino e teste
y_teste = base_teste.iloc[:, 0].values
y_teste

array([937750., 725126., 135000., ..., 402101., 400000., 325000.])

Os índices e os cabeçalhos precisam ser removidos do dataset. Outro ponto importante é que o XGBoost do SageMaker espera receber a varíavel target no começo do dataset, no caso Price precisa ficar no começo. Caso isso não seja respeitado, o SageMaker retorna um erro no treinamento. 

O SageMaker espera os arquivos no formato CSV para usar o XGBoost.

In [25]:
# Cria csv para treino e teste
base_treinamento.to_csv('house_prices_train_xgboost.csv', header = False, index = False)
base_teste.to_csv('house_prices_test_xgboost.csv', header = False, index = False)

# 4. Configuração do SageMaker

In [ ]:
import sagemaker
import boto3
from sagemaker import Session

In [ ]:
session = sagemaker.Session()
bucket = 'cursoawssagemaker'
subpasta_modelo = 'modelos/house-prices/xgboost'
subpasta_dataset = 'datasets/house-prices'
key_train = 'houses-train-data-xgboost'
key_test = 'houses-test-data-xgboost'
role = sagemaker.get_execution_role()
s3_train_data = 's3://{}/{}/train/{}'.format(bucket, subpasta_dataset, key_train)
s3_test_data = 's3://{}/{}/test/{}'.format(bucket, subpasta_dataset, key_test)
output_location = 's3://{}/{}/output'.format(bucket, subpasta_modelo)
print('Role: {}'.format(role))
print('Localização da base de treinamento: {}'.format(s3_train_data))
print('Localização da base de teste: {}'.format(s3_test_data))
print('Modelo final será salvo em: {}'.format(output_location))

In [ ]:
import os
with open('house_prices_train_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'train', key_train)).upload_fileobj(f)

In [ ]:
with open('house_prices_test_xgboost.csv', 'rb') as f:
    boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(subpasta_dataset, 'test', key_test)).upload_fileobj(f)

# 5. Treinamento do XGBoost

# 6. Avaliação do Modelo

# 7. Tunning do Modelo

# 8. Novo modelo com Tunning